In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-20-015-mixed-00409/translator1/fingerprint.pb
/kaggle/input/final-20-015-mixed-00409/translator1/saved_model.pb
/kaggle/input/final-20-015-mixed-00409/translator1/variables/variables.index
/kaggle/input/final-20-015-mixed-00409/translator1/variables/variables.data-00000-of-00001
/kaggle/input/hin-vocab/vocab_data.pkl
/kaggle/input/transl-last-hour/translator1/fingerprint.pb
/kaggle/input/transl-last-hour/translator1/saved_model.pb
/kaggle/input/transl-last-hour/translator1/variables/variables.index
/kaggle/input/transl-last-hour/translator1/variables/variables.data-00000-of-00001
/kaggle/input/validate-bleu/validation.tsv
/kaggle/input/final-20-01-15e/translator1/fingerprint.pb
/kaggle/input/final-20-01-15e/translator1/saved_model.pb
/kaggle/input/final-20-01-15e/translator1/variables/variables.index
/kaggle/input/final-20-01-15e/translator1/variables/variables.data-00000-of-00001
/kaggle/input/final-20-02-15e/translator1/fingerprint.pb
/kaggle/input/final-20-02-15e

In [3]:
from nltk.translate.bleu_score import sentence_bleu

In [4]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.7 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [5]:
import tensorflow as tf
import tensorflow_text as tf_text

In [6]:
!pip install Metaphone
from metaphone import doublemetaphone

  Preparing metadata (setup.py) ... done
  Created wheel for Metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13919 sha256=b6e7c44a24cb100e7afd89c7dba885d26e9d979819c9cb66ff79a8309d5ffc84
  Stored in directory: /root/.cache/pip/wheels/3a/2c/57/cb7d38aa6c80a870a0341ccf4265c2921fb55b1f7f4b76b46b
Successfully built Metaphone


In [7]:
import pickle
with open('/kaggle/input/hin-vocab/vocab_data.pkl', 'rb') as fp:
    hin_vocab = pickle.load(fp)
vocab_keys=[l for l in hin_vocab]
len(hin_vocab),len(vocab_keys)

(26214, 26214)

In [8]:
import re
re.sub("[.!?\\-\'\"]", "", "    kaise ho tum viase .?,KSDFaha   ").lower().strip()

'kaise ho tum viase ,ksdfaha'

In [9]:
def t_text(line):
    line=re.sub("[.!?\\-\'\"]", "",line).lower().strip()
    string=''
    for j in line.split(' '):
        if doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:] in vocab_keys:
            string=string+list(hin_vocab[doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:]])[0]+' '
        else:
            string=string+j+' '
    return string.lower().strip()

In [10]:
import pandas as pd
data_val=pd.read_csv('/kaggle/input/validate-bleu/validation.tsv',sep='\t')
data_val=data_val[['cs_query','en_query']]
data_val=data_val.rename(columns={'cs_query':'Sentence','en_query':'trr'})
data_val

,Sentence,trr
0,mere timer ko roko,Pause my timer .
1,Please Liz ke liye video message banaye,Please make video message for Liz
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday
3,Library events Cedar Park me hai,Library events in Cedar Park
4,Please timer ko rokey,Please pause timer
...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,I ' d like you to snooze my alarm for 10 minut...
1387,please Tupac play kare,Please play Tupac
1388,Rhonda ko message kare aur July me Vacation Bi...,Message Rhonda Ask her details about Vacation ...


In [11]:
'''three_input_text = tf.constant([t_text('sabse best friend'), 
                                t_text('That toh I know'),
                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhara papa ka naam kya hai'),
])

result = reloaded.translate(input_text=three_input_text)

for tr in result['text']:
  print(tr.numpy().decode())'''

"three_input_text = tf.constant([t_text('sabse best friend'), \n                                t_text('That toh I know'),\n                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),\n                                t_text('tum batao'),\n                                t_text('kya karna hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhara papa ka naam kya hai'),\n])\n\nresult = reloaded.translate(input_text=three_input_text)\n\nfor tr in result['text']:\n  print(tr.numpy().decode())"

In [11]:
import numpy

In [12]:
three_input_text = tf.constant([#'sabse best friend', 
                                #'That toh I know',
                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'she feels said the director s fair . . the f of the poor call'
 b'you will be' b'whatever you want to get a ! couple !'], shape=(3,), dtype=string)
she feels said the director s fair . . the f of the poor call
you will be
whatever you want to get a ! couple !


In [13]:
import numpy
reloaded.tf_translate(
            tf.constant([
                t_text('tumhare papa ka naam kya hai')
                        ]))['text'][0].numpy().decode()

'o dear . . . by cutting a child work . . .'

In [12]:
def cal_bleu(x,y):
    return sentence_bleu(x.split(),y.split())

In [22]:
reloaded = tf.saved_model.load('/kaggle/input/transl-last-hour/translator1')
data_val['model_output1']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode())
data_val['bleu_score1']=data_val[['Sentence','model_output1']].apply(lambda x: cal_bleu(*x),axis=1)

In [17]:
data_val

,Sentence,trr,model_output1,bleu_score1
0,mere timer ko roko,Pause my timer .,my timer muffins run,0.000000
1,Please Liz ke liye video message banaye,Please make video message for Liz,please give the liz mubarak mia . . . i have w...,0.526640
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,"shall i remain at pm and pm on tuesday , on tu...",0.000000
3,Library events Cedar Park me hai,Library events in Cedar Park,temperate in cedar park cedar events,0.000000
4,Please timer ko rokey,Please pause timer,please pause timer needs to be delayed .,0.000000
...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,outdoor outdoor concerts this summer,0.000000
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,I ' d like you to snooze my alarm for 10 minut...,i am that you are a four time ago toy .,0.722657
1387,please Tupac play kare,Please play Tupac,please play tupac on advance .,0.000000
1388,Rhonda ko message kare aur July me Vacation Bi...,Message Rhonda Ask her details about Vacation ...,ask sri director for vacation bible school gab...,0.000000


In [18]:
sentence_bleu("i have eaten".split(), "i have eaten the food".split() )

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.668740304976422

In [23]:
data_val['bleu_score1'].mean()

0.19700888898053723

In [24]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-015-mixed-00409/translator1')
data_val['model_output2']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode())
data_val['bleu_score2']=data_val[['Sentence','model_output2']].apply(lambda x: cal_bleu(*x),axis=1)

In [25]:
data_val['bleu_score2'].mean()

0.2246556751247553

In [26]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-02-15e/translator1')
data_val['model_output3']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode())
data_val['bleu_score3']=data_val[['Sentence','model_output3']].apply(lambda x: cal_bleu(*x),axis=1)

In [27]:
data_val['bleu_score3'].mean()

0.20781863076996535

In [14]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-01-15e/translator1')
data_val['model_output4']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode())
data_val['bleu_score4']=data_val[['Sentence','model_output4']].apply(lambda x: cal_bleu(*x),axis=1)

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [15]:
data_val['bleu_score4'].mean()

0.18146761694325464

In [16]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-15e/translator')
data_val['model_output5']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode())
data_val['bleu_score5']=data_val[['Sentence','model_output5']].apply(lambda x: cal_bleu(*x),axis=1)

In [17]:
data_val['bleu_score5'].mean()

0.20222197851064017

In [18]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-22e/translator')
data_val['model_output6']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode())
data_val['bleu_score6']=data_val[['Sentence','model_output6']].apply(lambda x: cal_bleu(*x),axis=1)

In [19]:
data_val['bleu_score6'].mean()

0.18771130403523018

In [20]:
reloaded = tf.saved_model.load('/kaggle/input/translator1/translator1')
data_val['model_output7']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode())
data_val['bleu_score7']=data_val[['Sentence','model_output7']].apply(lambda x: cal_bleu(*x),axis=1)

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [21]:
data_val['bleu_score7'].mean()

0.19088513850951083

In [28]:
reloaded = tf.saved_model.load('/kaggle/input/translator-2/translator1')
data_val['model_output8']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode())
data_val['bleu_score8']=data_val[['Sentence','model_output8']].apply(lambda x: cal_bleu(*x),axis=1)

In [29]:
data_val['bleu_score8'].mean()

0.19885362733151254

In [30]:
data_val

,Sentence,trr,model_output4,bleu_score4,model_output5,bleu_score5,model_output6,bleu_score6,model_output7,bleu_score7,model_output1,bleu_score1,model_output2,bleu_score2,model_output3,bleu_score3,model_output8,bleu_score8
0,mere timer ko roko,Pause my timer .,my timer will go,0.000000,pause my timer .,0.000000,pause my timer,0.000000,pause my timer .,0.000000,pause my timer .,0.000000,stop my timer .,0.000000,pause my timer at the gym,0.000000,pause my timer .,0.000000
1,Please Liz ke liye video message banaye,Please make video message for Liz,please provide a liz from the liz of liz . you...,0.604275,please help video message to liz s of plan tec...,0.549100,liz s copy video message please to liz s care ...,0.638943,please transfer the beginning of the storm pre...,0.000000,"please set a reminder of onions , she will be ...",0.516973,please make an message for the beginning of th...,0.000000,please message liz granny s granny s not open ...,0.652994,please apply by modis s house laugh . . .,0.562341
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,do abhishek show audience ke . pm and pm . usu...,0.000000,instead of strike for years sir at pm and pm o...,0.000000,indias doctor has a lot made at pm and pm tues...,0.537285,madam anthem they are at pm and pm on tuesday ...,0.000000,report for a reminder to pt . shri . pm and pm...,0.508133,have you wearing the underwear create been als...,0.000000,alarm the sound of change at pm and pm on tuesday,0.000000,change the reminder about the flooding two fro...,0.000000
3,Library events Cedar Park me hai,Library events in Cedar Park,at library events related to the library in te...,0.000000,the library events there can be at the library...,0.000000,in the library events for families .,0.000000,library events cedar park performing,0.000000,library events cedar park performing,0.000000,library events at the childrens events meant t...,0.537285,library events in cedar ia nc,0.000000,library events cedar park performing,0.000000
4,Please timer ko rokey,Please pause timer,please stop the timer only minutes,0.000000,please pause timer .,0.000000,please pause timer .,0.000000,please pause timer remaining,0.000000,please pause timer needs .,0.000000,please pause the timer,0.000000,please pause timer once,0.000000,please pause timer remaining,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,play upcoming summer sleep by jain,0.000000,disturbed concerts this summer,0.000000,summer events in mesquite this summer,0.000000,outdoor outdoor concerts this summer,0.000000,outdoor outdoor concerts this summer,0.000000,cheap summer channel,0.000000,hair used concerts this summer,0.000000,outdoor outdoor concerts this summer,0.000000
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,I ' d like you to snooze my alarm for 10 minut...,i would like you to change my dirty blow . . ....,0.567128,i want you to remind my brother . . the standa...,0.569599,i want you to the burning of your king behind ...,0.500000,i am a kiss you . . . . .,0.740083,"i am done on the way , you have to snooze my s...",0.531830,i may ! you will snooze my chicken for my cher...,0.626284,"i like you , my alarm will stop tweeting lodge...",0.614788,it became a kiss to me . . i will snooze my hu...,0.614788
1387,please Tupac play kare,Please play Tupac,please play no tupac of the pittsburgh,0.000000,please play tupac,0.000000,tupac play . please tupac,0.000000,please play tupac play given .,0.000000,please play tupac s help ?,0.638943,please play tupac play . please,0.000000,please play the tupac at the mat .,0.000000,please play tupac ?,0.000000
1388,Rhonda ko message kare aur July me Vacation Bi...,Message Rhonda Ask her details about Vacation ...,under hope of the temperature in the morning a...,0.000000,message estimate in four july bible school in ...,0.000000,message rhonda wednesday to